In [2]:
from pycalphad import Database, calculate
import numpy as np

dbf = Database("Al-Cu-Y.tdb")
components = ["AL", "CU", "Y", "VA"]
phases = list(dbf.phases.keys())

In [3]:
pressures = np.linspace(0, 1e3, 11)
pressures

array([   0.,  100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,
        900., 1000.])

In [4]:
temperatures = np.linspace(0, 1e3, 11)
temperatures

array([   0.,  100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,
        900., 1000.])

## Get All combinations of components
We define a function that is useful when calculating properties over all combinations of components in a thermodynamic system. It generates all possible combinations - from individual elements to multiple-element sets. The result is a power set of components (excluding the empty set), which allows us to create a diagram for each combination of components.

In [5]:
import itertools
from typing import List

def generate_self_cartesian(elements: List[str]) -> List[List[str]]:
    """
    Generates the Cartesian product of a given set with itself.

    Args:
        elements (List[str]): The list of elements for which the Cartesian product
        should be generated.

    Returns:
        List[List[str]]: A list of all pairs in the Cartesian product of the input
        elements with itself. Each pair is represented as a list of two strings.

    Example:
        >>> generate_self_cartesian(["AL", "CU", "Y", "VA"])
        [['AL'], ['CU'], ['Y'], ['VA'], ['AL', 'CU'], ['AL', 'Y'], ['AL', 'VA'], ['CU', 'Y'], ['CU', 'VA'], ['Y', 'VA'], ['AL', 'CU', 'Y'], ['AL', 'CU', 'VA'], ['AL', 'Y', 'VA'], ['CU', 'Y', 'VA'], ['AL', 'CU', 'Y', 'VA']]
    """
    combinations = []
    for r in range(1, len(elements) + 1):
        combinations.extend(itertools.combinations(elements, r))
    return [list(comb) for comb in combinations]

component_combos = generate_self_cartesian(["AL", "CU", "Y", "VA"])

In [9]:
from pycalphad.core.utils import filter_phases, unpack_components
results = []
output_properties = ["GM", "SM", "HM", "CPM"]
for combo in component_combos:
    species_obj_set = unpack_components(dbf, combo)
    list_of_possible_phases = filter_phases(dbf, species_obj_set)
    for phase in list_of_possible_phases:
        for temp in temperatures:
            for pressure in pressures:
                for output_property in output_properties:
                    c = calculate(dbf, species_obj_set, phase, output=output_property, T=temp, P=pressure)
                    if len(c.X.points) in [0, 1]:
                        continue
                    xs = c.X.values[0, 0, 0, :, 0]
                    ys = c.X.values[0, 0, 0, :, 1]
                    zs = c[output_property].values[0, 0, 0, :]
                    result = {
                        "phase": phase,
                        "temperature": temp,
                        "pressure": pressure,
                        "composition": combo,
                        "output_property": output_property,
                        "xs": xs,
                        "ys": ys,
                        "zs": zs,
                    }
                    results.append(result)

In [11]:
results

[{'phase': 'AL2Y',
  'temperature': 0.0,
  'pressure': 0.0,
  'composition': ['AL', 'CU'],
  'output_property': 'GM',
  'xs': array([1.        , 0.66667   , 0.33333   , ..., 0.57555143, 0.12545961,
         0.91906043]),
  'ys': array([1.00000000e-14, 3.33330000e-01, 6.66670000e-01, ...,
         4.24448569e-01, 8.74540393e-01, 8.09395719e-02]),
  'zs': array([5.00000000e+03, 5.00000000e-11, 5.00000000e-11, ...,
         1.86313463e+03, 2.97397610e+01, 3.90737387e+03])},
 {'phase': 'AL2Y',
  'temperature': 0.0,
  'pressure': 0.0,
  'composition': ['AL', 'CU'],
  'output_property': 'GM',
  'xs': array([1.        , 0.66667   , 0.33333   , ..., 0.57555143, 0.12545961,
         0.91906043]),
  'ys': array([1.00000000e-14, 3.33330000e-01, 6.66670000e-01, ...,
         4.24448569e-01, 8.74540393e-01, 8.09395719e-02]),
  'zs': array([5.00000000e+03, 5.00000000e-11, 5.00000000e-11, ...,
         1.86313463e+03, 2.97397610e+01, 3.90737387e+03])},
 {'phase': 'AL2Y',
  'temperature': 0.0,
  'pres

In [12]:
len(results)

30009